In [1]:
import os 
import logging
# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess


In [2]:
class YelpCrawler (scrapy.Spider):
    name = "yelp-spy"
    start_urls = ['https://www.yelp.fr/']
    
    def parse(self, response):
        key_desc = "reastaurant japonais"
        key_loc = "Paris"
        return scrapy.FormRequest.from_response(response, formdata={'find_desc':key_desc, 'find_loc':key_loc},
                                               callback=self.after_search)
        #pass
    
    def after_search(self ,response):
        #.css-1661a90
        rest_list = response.css('h4.css-1l5lt1i')
        for tag in rest_list:
            #print(tag)
            yield{
                'name':tag.css('span a::text').get(),
                #'url': tag.css('span a::attr(href)').getall() # PEZ: It creates a list with one element
                #'url': tag.css('span a::attr(href)').extract_first(),# PEZ: This works fine also!
                'url': tag.css('span a').attrib['href'], #PEZ: This works fine!
                'next-link': response.css('a.next-link').attrib['href']
            }
            try:
                # Identify what the next page is. type(next_page) is str
                next_page = response.css('a.next-link').attrib['href']
            except KeyError:
                logging.info('There is no next page. The crawling process will terminate')
            else:
                # if the next page is found
                yield response.follow(next_page, callback = self.after_search)

In [3]:
def remove_file(filename):
    if filename in os.listdir("data/"):
        os.remove("data/" + filename)

In [4]:
filename = "restaurant_japonais-paris.json"
remove_file(filename)
process = CrawlerProcess(settings={
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'data/' + filename : {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True  # AutoThrottle Here!
})


2021-07-04 15:01:35 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-07-04 15:01:35 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-5.4.89+-x86_64-with-glibc2.10


In [5]:
process.crawl(YelpCrawler)
process.start()

2021-07-04 15:01:35 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-07-04 15:01:35 [scrapy.extensions.telnet] INFO: Telnet Password: f32edee4ed188d5c
2021-07-04 15:01:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-07-04 15:01:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.down